In [1]:
import pandas as pd

In [2]:
training = pd.read_json("data/training_set.json")
data = training[:1000]

In [3]:
from processing.utils import filter_data_by_properties,select_structures

/home/jkdamewo/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
data = filter_data_by_properties(data,"dft_e_hull")

In [5]:
data_unrelaxed = select_structures(data,"unrelaxed")

In [6]:
data_relaxed = select_structures(data,"relaxed")

In [7]:
from processing.interpolation.Interpolation import *

In [8]:
data_unrelaxed = apply_interpolation(data_unrelaxed,"dft_e_hull")

In [9]:
data_unrelaxed

,formula,framework,composition,n_atoms_unrelaxed,n_atoms_opt,unrelaxed_cryst_id,unrelaxed_struct,opt_cryst_id,opt_struct,es_job_id,dft_energy,dft_energy_per_atom,dft_e_hull,structure,dft_e_hull_interp,dft_e_hull_diff
0,Ba1Ca7Fe3Mn5O24,Ba0.125Ca0.875Fe0.375Mn0.625O3,"{'sites': {'A': ['Ca', 'Ba'], 'B': ['Mn', 'Fe'...",40,40,222996501,"{'@module': 'pymatgen.core.structure', '@class...",252537945,"{'@module': 'pymatgen.core.structure', '@class...",72945546,-266.633305,-6.665833,0.122559,"[[8.e-06 8.e-06 8.e-06] Ba, [2.000008 2.000008...",0.140324,-0.017766
1,Ba1Ca7Fe3Mn5O24,Ba0.125Ca0.875Fe0.375Mn0.625O3,"{'sites': {'A': ['Ca', 'Ba'], 'B': ['Mn', 'Fe'...",40,40,222996502,"{'@module': 'pymatgen.core.structure', '@class...",253437540,"{'@module': 'pymatgen.core.structure', '@class...",73304197,-266.521519,-6.663038,0.125353,"[[0. 0. 0.] Ba, [2. 2. 2.] Fe, [2. 2. 0.] O, [...",0.140324,-0.014971
2,Ba1Cr4Fe4O24Pr7,Ba0.125Pr0.875Cr0.500Fe0.500O3,"{'sites': {'A': ['Ba', 'Pr'], 'B': ['Cr', 'Fe'...",40,40,201883190,"{'@module': 'pymatgen.core.structure', '@class...",203132378,"{'@module': 'pymatgen.core.structure', '@class...",61473323,-300.739117,-7.518478,0.189088,"[[8.e-06 8.e-06 8.e-06] Ba, [8.000000e-06 8.00...",0.387373,-0.198285
3,Ba1Cr4Fe4O24Pr7,Ba0.125Pr0.875Cr0.500Fe0.500O3,"{'sites': {'A': ['Ba', 'Pr'], 'B': ['Cr', 'Fe'...",40,40,200882398,"{'@module': 'pymatgen.core.structure', '@class...",201610028,"{'@module': 'pymatgen.core.structure', '@class...",61473401,-302.920491,-7.573012,0.134554,"[[8.e-06 8.e-06 8.e-06] Ba, [8.000000e-06 8.00...",0.387373,-0.252819
4,Ba1Co3Ni5O24Sm7,Ba0.125Sm0.875Co0.375Ni0.625O3,"{'sites': {'A': ['Ba', 'Sm'], 'B': ['Co', 'Ni'...",40,40,201883295,"{'@module': 'pymatgen.core.structure', '@class...",255717779,"{'@module': 'pymatgen.core.structure', '@class...",74141778,-261.814429,-6.545361,0.158405,"[[0. 0. 0.] Ba, [0. 0. 4.] Sm, [0.0000000e+00 ...",0.223073,-0.064669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Ba8Ga4Nb4O24,BaGa0.500Nb0.500O3,"{'sites': {'A': ['Ba'], 'B': ['Ga', 'Nb'], 'X'...",40,40,261447638,"{'@module': 'pymatgen.core.structure', '@class...",261595176,"{'@module': 'pymatgen.core.structure', '@class...",76518029,-288.738648,-7.218466,0.071669,"[[6.000008 2.000008 2.000008] Ba, [8.000000e-0...",0.118783,-0.047114
996,Ba8Ga4Nb4O24,BaGa0.500Nb0.500O3,"{'sites': {'A': ['Ba'], 'B': ['Ga', 'Nb'], 'X'...",40,40,261447639,"{'@module': 'pymatgen.core.structure', '@class...",261610387,"{'@module': 'pymatgen.core.structure', '@class...",76521316,-288.594702,-7.214868,0.075268,"[[6.000008 2.000008 2.000008] Ba, [8.000000e-0...",0.118783,-0.043515
997,Ba8Ga4Nb4O24,BaGa0.500Nb0.500O3,"{'sites': {'A': ['Ba'], 'B': ['Ga', 'Nb'], 'X'...",40,40,261447641,"{'@module': 'pymatgen.core.structure', '@class...",261595087,"{'@module': 'pymatgen.core.structure', '@class...",76518119,-288.751204,-7.218780,0.071356,"[[6.000008 2.000008 6.000008] Ba, [8.000000e-0...",0.118783,-0.047428
998,Ba8Ga4O24Ta4,BaGa0.500Ta0.500O3,"{'sites': {'A': ['Ba'], 'B': ['Ga', 'Ta'], 'X'...",40,40,261447648,"{'@module': 'pymatgen.core.structure', '@class...",261838380,"{'@module': 'pymatgen.core.structure', '@class...",76546918,-301.049211,-7.526230,0.041065,"[[6.000008 6.000008 2.000008] Ba, [8.000000e-0...",0.138295,-0.097229


In [10]:
from processing.dataloader.dataloader import get_dataloader
from processing.create_model.create_model import create_model
from training.hyperparameters.default import get_default_cgcnn_hyperparameters, get_default_painn_hyperparameters, get_default_e3nn_hyperparameters
from training.model_training.trainer import trainer

In [ ]:
train_loader = get_dataloader(data_unrelaxed,"dft_e_hull","CGCNN",4)
model, Normalizer = create_model("CGCNN",train_loader)

In [ ]:
hyperparameters = get_default_cgcnn_hyperparameters()
print(hyperparameters)
best_model, loss_fn = trainer(model,Normalizer,'CGCNN',train_loader,train_loader,hyperparameters,"saved_models/CGCNN/Scrap",0)

In [ ]:
train_loader = get_dataloader(data_unrelaxed,"dft_e_hull","Painn",4)
model, Normalizer = create_model("Painn",train_loader)

In [14]:
hyperparameters = get_default_painn_hyperparameters()
best_model, loss_fn = trainer(model,Normalizer,'Painn',train_loader,train_loader,hyperparameters,"saved_models/Painn/Scrap",0)


 Time | Epoch | Learning rate | Train loss | Validation loss | MAE_dft_e_hull_diff | GPU Memory (MB)


100%|█████████▉| 249/250 [00:05<00:00, 43.17it/s]


36:15 |     7 |     1.000e-03 |     0.0463 |          0.0576 |              0.0576 |             129


100%|█████████▉| 249/250 [00:05<00:00, 42.87it/s]


36:22 |     8 |     1.000e-03 |     0.0440 |          0.0470 |              0.0470 |             129


...

36:45 |    11 |     1.000e-03 |     0.0391 |          0.0422 |              0.0422 |             129


100%|█████████▉| 249/250 [00:05<00:00, 43.47it/s]


36:52 |    12 |     1.000e-03 |     0.0375 |          0.0494 |              0.0494 |             129


 88%|████████▊ | 221/250 [00:05<00:00, 42.19it/s]


KeyboardInterrupt: 

In [15]:
train_loader = get_dataloader(data_unrelaxed,"dft_e_hull","e3nn",4)
model, Normalizer = create_model("e3nn",train_loader)

100%|██████████| 1000/1000 [00:36<00:00, 27.07it/s]
/home/jkdamewo/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


In [16]:
hyperparameters = get_default_e3nn_hyperparameters()
print(hyperparameters)
best_model, loss_fn = trainer(model,Normalizer,'e3nn',train_loader,train_loader,hyperparameters,"saved_models/e3nn/Scrap",0)

{'MaxEpochs': 100, 'log_lr': -3, 'reduceLR_patience': 10, 'len_embedding_feature_vector': 64, 'num_hidden_feature': 128, 'num_hidden_layer': 2, 'multiplicity_irreps': 32, 'num_conv': 3, 'num_radical_basis': 10, 'num_radial_neurons': 100}


100%|██████████| 250/250 [00:21<00:00, 11.38it/s]


   1 ,lr = 0.00100000  train loss = 0.05213978  val loss = 0.05213978  time = 00:00:21


100%|██████████| 250/250 [00:20<00:00, 12.31it/s]


   2 ,lr = 0.00100000  train loss = 0.04795978  val loss = 0.04795978  time = 00:00:20


 20%|█▉        | 49/250 [00:03<00:16, 12.47it/s]


KeyboardInterrupt: 

In [11]:
train_loader = get_dataloader(data_unrelaxed,"dft_e_hull","e3nn_contrastive",1)
model, Normalizer = create_model("e3nn_contrastive",train_loader)

100%|██████████| 1000/1000 [00:00<00:00, 3402.82it/s]
/home/jkdamewo/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


In [12]:
hyperparameters = get_default_e3nn_hyperparameters()
print(hyperparameters)
best_model, loss_fn = trainer(model,Normalizer,'e3nn_contrastive',train_loader,train_loader,hyperparameters,"saved_models/e3nn_contrastive/Scrap",0)

{'MaxEpochs': 100, 'log_lr': -3, 'reduceLR_patience': 10, 'len_embedding_feature_vector': 64, 'num_hidden_feature': 128, 'num_hidden_layer': 2, 'multiplicity_irreps': 32, 'num_conv': 3, 'num_radical_basis': 10, 'num_radial_neurons': 100}


100%|██████████| 245/245 [00:22<00:00, 10.75it/s]


   1 ,lr = 0.00100000  train loss = 0.08280612  train direct = 14.09573240  train contrastive = 5.71330795  val loss = 0.08280612  val direct = 14.09573240  val contrastive = 5.71330795  time = 00:00:22


100%|██████████| 245/245 [00:19<00:00, 12.54it/s]


   2 ,lr = 0.00100000  train loss = 0.07947576  train direct = 13.29255131  train contrastive = 5.71307829  val loss = 0.07947576  val direct = 13.29255131  val contrastive = 5.71307829  time = 00:00:19


...

  15 ,lr = 0.00100000  train loss = 0.04868155  train direct = 6.65114205  train contrastive = 4.79544613  val loss = 0.04868155  val direct = 6.65114205  val contrastive = 4.79544613  time = 00:00:19


100%|██████████| 245/245 [00:19<00:00, 12.35it/s]


  16 ,lr = 0.00100000  train loss = 0.05074150  train direct = 7.21516263  train contrastive = 4.73276732  val loss = 0.05074150  val direct = 7.21516263  val contrastive = 4.73276732  time = 00:00:19


100%|██████████| 245/245 [00:19<00:00, 12.45it/s]


KeyboardInterrupt: 

In [30]:
#### Looking at contrastive dataloader
from tqdm import tqdm

batch_num = 7

for j, d in tqdm(enumerate(train_loader), total=len(train_loader)):
    
    if j == 4:
        
        print(d)
        print(d.comp)


100%|██████████| 62/62 [00:00<00:00, 489.00it/s]

DataCompBatch(x=[880, 92], edge_index=[2, 44880], pos=[880, 3], lattice=[22, 3, 3], symbol=[22], comp=[22], z=[880, 92], edge_shift=[44880, 3], edge_vec=[44880, 3], edge_len=[22], target=[22, 1], idx=[22, 1], batch=[880], ptr=[23])
['La4Mn8O24Sr4', 'La4Mn8O24Sr4', 'La4Mn8O24Sr4', 'La4Mn8O24Sr4', 'La4Mn8O24Sr4', 'Mg5Mn4O24Sr3Ta4', 'Mg5Mn4O24Sr3Ta4', 'Mg5Mn4O24Sr3Ta4', 'Mg5Mn4O24Sr3Ta4', 'Mg5Mn4O24Sr3Ta4', 'Mg5Mn4O24Sr3Ta4', 'Co4Mn4O24Pr8', 'Co4Mn4O24Pr8', 'Co4Mn4O24Pr8', 'Co4Mn4O24Pr8', 'Co4Mn4O24Pr8', 'Co4Fe4La2O24Sr6', 'Co4Fe4La2O24Sr6', 'Co4Fe4La2O24Sr6', 'Co4Fe4La2O24Sr6', 'Co4Fe4La2O24Sr6', 'Co4Fe4La2O24Sr6']
